# spotlob


Version 0.9.0a

---

Python meetup 18.04.2019


Fabian Meyer

Fraunhofer ISE

# Task: image segmentation

* Find features in scientific image data
* Precise measurement
* Traceable and repeatable evaluation
* no training data available
* Semi-automatic: tweaking will be needed


<img src="presentation-images/usecases.png">

# There is very mature software for this...


<br>
<table>
    <tr>
        <td>
            <img src="presentation-images/opencv.png" height="200" width="200">
        </td>
        <td>
            <img src="presentation-images/scikit-image.png" height="200" width="200">
        </td>
    </tr>  
    <tr>
        <td style="text-align:left; font-size:16pt">
            <h3>OpenCV</h3>
            <ul>
                <li>C/C++ with Python interface</li>
                <li>fast</li>
                <li>also very popular outside Python world</li>
                <li>not exactly pythonic to use</li>
            </ul>  
        </td>
        <td style="text-align:left; font-size:16pt; vertical-align:top">
            <h3>Scikit-image</h3>
            <ul>
                <li>Python/Cython</li>
                <li>feature-rich</li>
                <li>integrates well with other popular python packages</li>
                <li>great documentation</li>
            </ul>
        </td>
    </tr>  
</table>

# Why spotlob?

## Typical workflow

1. record a new set of images
2. search for an old script to evaluate them
3. doesn't work
4. search for another script to evaluate them
3. doesn't work
4. merge the scripts and make it work by iteratively changing parameters
5. apply script on whole set of images
6. memory full

## Two months later: next set of images
start all over

# Why spotlob?

* finding the right detection quicker
* standardize for later use
* don't lose flexibility compared to scripting
* jupyter compatible
* low memory consumption
* multithreaded batch processing
* combine multiple libraries

# Notebook demo

In [ ]:
%matplotlib widget

In [ ]:
import sys
sys.path.append("../")

In [ ]:
from spotlob.defaults import *

gui = make_gui("presentation-images/demoim.jpg")
show_gui(gui)

<img src="presentation-images\pipeline.png">

# Detection process steps

# Conversion

<img src="presentation-images/hsv.png" width="800px" height="600px">

<img src="presentation-images\pipeline.png">

# Detect features - the pipeline

1. Load image as array
2. Extract a single channel
4. Binarization
6. Find connected areas
7. Evaluate them

# Results as pandas dataframes

In [ ]:
gui.results()

# The pipeline

In [ ]:
print(gui.pipeline)

# The pipeline

* the sequence of processes to apply, ProcessSteps
* the parameters

= everything you need to know to repeat the detection

# Store the pipeline

In [ ]:
gui.pipeline.save("mypipe.pipe")

# and restore

In [ ]:
from spotlob.pipeline import Pipeline

mypipe = Pipeline.from_file("mypipe.pipe")

print(mypipe)

# batch processing

In [ ]:
from glob import glob

all_images = glob("*.JPG")

In [ ]:
from spotlob.batch import batchprocess

all_results = batchprocess("mypipe.pipe", all_images)

In [ ]:
all_results.groupby("filepath").agg(["mean", "std"])

# Change the pipeline

another binarization function

In [ ]:
from spotlob.process_opencv import BinaryThreshold

binarization = BinaryThreshold(threshold = 100)

new_pipeline = gui.pipeline.replaced_with(binarization)

print(new_pipeline)

# Use the new pipeline

In [ ]:
gui = make_gui("presentation-images/demoim.jpg")

gui.pipeline = new_pipeline

show_gui(gui)

# Extend spotlob: subclassing

In [ ]:
from spotlob.process_steps import Binarization

class MyThreshold(Binarization):
    
    def __init__(self, threshold):
        threshold_parameter = NumericRangeParameter("threshold", threshold, 0, 255)
        
        pars = SpotlobParameterSet([threshold_parameter])
        
        super(BinaryThreshold, self).__init__(self.threshold_fn, pars)

    def my_threshold_function(self, grey_image, threshold):
        _, im = cv2.threshold(grey_image, threshold, 255, cv2.THRESH_BINARY)
        return im

# Extend spotlob: with a function...

In [ ]:
import numpy as np

def my_threshold(image, lower_threshold, upper_threshold):
    above_lower = image > lower_threshold
    below_upper = image < upper_threshold
    
    out = np.logical_and(above_lower,
                         below_upper).astype(np.uint8)*255
    return out

# Extend spotlob: ...using decorators

In [ ]:
from spotlob.register import PROCESS_REGISTER as register

gui = make_gui("presentation-images/demoim.jpg")

@use_in(gui)
@register.binarization_plugin([("lower_threshold",(0,255,100)),
                               ("upper_threshold",(0,255,200))])
def my_threshold(image, lower_threshold, upper_threshold):
    above_lower = image > lower_threshold
    below_upper = image < upper_threshold
    
    out = np.logical_and(above_lower,
                         below_upper).astype(np.uint8)*255
    return out

# `@use_in(gui)`

In [ ]:
%matplotlib widget
show_gui(gui)

# jupyter.ise.fhg.de

use kernel `Python 3.6.3`

# Installation

```
pip install spotlob
```

# Source & docs

```
https://gitlab.cc-asp.fraunhofer.de/fmeyer/spotlob
```

# Thank you
## for your attention

# Questions
# Suggestions